### Data 2021

In [1]:
import pandas as pd
import numpy as np

In [ ]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2021" # fetching all the 2020 movies from wikipedia
df1 = pd.read_html(link, header=0)[2]
df2 = pd.read_html(link, header=0)[3]
df3 = pd.read_html(link, header=0)[4]
df4 = pd.read_html(link, header=0)[5]

In [4]:
df = pd.concat([df1,df2,df3,df4],ignore_index=True) #concat all the dataframes in a single for preprocessing
df

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,J A N U A R Y,1.0,Shadow in the Cloud,Vertical Entertainment,Roseanne Liang (director/screenplay); Max Land...,[2]
1,J A N U A R Y,5.0,Hacksaw,Leone Films / Midnight Releasing,"Anthony Leone (director/screenplay); Amy Cay, ...",[3]
2,J A N U A R Y,12.0,Dr. Bird's Advice for Sad Poets,Relativity Media / Ketchup Entertainment,Yaniv Raz (director/screenplay); Lucas Jade Zu...,[4]
3,J A N U A R Y,13.0,The White Tiger,Netflix / ARRAY / Purple Pebble Pictures,Ramin Bahrani (director/screenplay); Adarsh Go...,NaN
4,J A N U A R Y,14.0,Locked Down,HBO Max / Warner Bros. Pictures,Doug Liman (director); Steven Knight (screenpl...,[5]
...,...,...,...,...,...,...
361,D E C E M B E R,25.0,The Tragedy of Macbeth,Apple TV+ / A24 / IAC Films,Joel Coen (director/screenplay); Denzel Washin...,[278]
362,D E C E M B E R,25.0,A Journal for Jordan,Columbia Pictures / Escape Artists / Bron Studios,Denzel Washington (director); Virgil Williams ...,[279]
363,D E C E M B E R,25.0,American Underdog,Lionsgate,"Erwin brothers (directors); Jon Erwin, David A...",[280]
364,D E C E M B E R,26.0,Memoria,Neon,Apichatpong Weerasethakul (director/acreenplay...,[281]


#### As it is observed in the above table, we do not have the genre column so we will get the genre data from IMDb. You will need to create an API key in order to access the API key.

### pip install tmdbv3api

In [5]:
#get your API Key here: https://www.themoviedb.org/settings/api
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()

api = 'cf1e5b9d8cf31e19913dd6bc256abb2a'
tmdb.api_key=api # insert your api key here


In [6]:
import requests

from tmdbv3api import Movie

tmdb_movie = Movie()

def get_genre(x):
    genres = []
    
    try:
        # Search for the movie title
        result = tmdb_movie.search(x)
        
        # Check if any results were found
        if not result:
            return 'No results found'

        # Get the movie ID from the first result
        movie_id = result[0].id
        
        # Fetch movie details using the movie ID
        response = requests.get(f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb.api_key}')
        data_json = response.json()
        
        # Check if the movie has genres
        if 'genres' in data_json and data_json['genres']:
            for genre in data_json['genres']:
                genres.append(genre['name'])
            return ' '.join(genres)  # Return genres as a space-separated string
        else:
            return np.nan
    
    except Exception as e:
        # Handle any exceptions (e.g., connection issues, API errors)
        return f'Error: {str(e)}'

# Example usage to map genres to the DataFrame
df['genres'] = df['Title'].map(lambda x: get_genre(x))


In [14]:
df['Cast and crew'].fillna('unknown',inplace=True)

In [15]:
df['Cast and crew'].isna().sum()

np.int64(0)

In [16]:
import re


def extract_info(text):
    # Initialize the director to None
    director_name = None
    # Initialize the actors to None
    actor_1, actor_2, actor_3 = None, None, None
    
    # Check if the text contains a director's name and role(s)
    director_match = re.search(r'([a-zA-Z\s]+) \(([^)]+)\)', text)  # Extract name and roles inside parentheses
    if director_match:
        name = director_match.group(1).strip()  # Extract the name (e.g., Michael Moore)
        roles = director_match.group(2).strip()  # Extract the roles (e.g., director/screenplay/narrator)
        
        # If 'director' is in the roles, assign only the name to the director column
        if 'director' in roles:
            director_name = name  # Only assign the name to the director column if 'director' is present in the roles
            
            # Remove the director's name from the actors list to avoid duplication
            text = text.replace(name + " (" + roles + ")", "")  # Remove the entire director's entry

    # Extract actors from the cast and crew field (after the last semicolon)
    actors = re.split(r';', text)[-1].strip().split(', ')  # Split by semicolon and take actors from the last part
    
    # Remove the director from the actors list (if it was listed there)
    actors = [actor for actor in actors if actor != director_name]
    
    # Assign actors if they exist
    actor_1 = actors[0] if len(actors) > 0 else None
    actor_2 = actors[1] if len(actors) > 1 else None
    actor_3 = actors[2] if len(actors) > 2 else None

    # Return the extracted information in a pandas Series
    return pd.Series([director_name, actor_1, actor_2, actor_3])



In [17]:
df[['director_name', 'actor_1_name', 'actor_2_name', 'actor_3_name']] = df['Cast and crew'].apply(extract_info)


In [25]:

df=df.rename(columns={'Title':'movie_title'})
df.sample(2)

,Opening,Opening.1,movie_title,Production company,Cast and crew,Ref.,genres,director_name,actor_1_name,actor_2_name,actor_3_name
140,J U N E,18.0,Rita Moreno: Just a Girl Who Decided to Go for It,Roadside Attractions,Mariem Pérez Riera (director/screenplay); Rita...,NaN,Documentary,rez Riera,Rita Moreno,Eva Longoria,George Chakiris
161,J U L Y,2.0,The Forever Purge,Universal Pictures / Platinum Dunes / Blumhous...,Everardo Gout (director); James DeMonaco (scre...,[116],Horror Action Thriller,Everardo Gout,Ana de la Reguera,Tenoch Huerta,Josh Lucas


In [26]:
new_df21 = df[['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]
new_df21.sample(2)

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
316,Manuel Miranda,Andrew Garfield,Alexandra Shipp,Robin de Jesús,Drama Music,"Tick, Tick... Boom!"
149,Josh Ruben,Sam Richardson,Milana Vayntrub,George Basil,Horror Comedy Fantasy,Werewolves Within


In [27]:
new_df21.isna().sum()

director_name     3
actor_1_name      0
actor_2_name      9
actor_3_name     28
genres            0
movie_title       1
dtype: int64

In [28]:
new_df21.fillna('unknown',inplace=True)

C:\Users\ARMS\AppData\Local\Temp\ipykernel_10792\1619479508.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df21.fillna('unknown',inplace=True)


In [29]:
new_df21.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
dtype: int64

In [30]:
new_df21['movie_title']= new_df21['movie_title'].str.lower()

C:\Users\ARMS\AppData\Local\Temp\ipykernel_10792\3483677005.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df21['movie_title']= new_df21['movie_title'].str.lower()


In [31]:

new_df21['comb'] = new_df21['actor_1_name'] + ' ' + new_df21['actor_2_name'] + ' '+ new_df21['actor_3_name'] + ' '+ new_df21['director_name'] +' ' + new_df21['genres']
new_df21.head()

C:\Users\ARMS\AppData\Local\Temp\ipykernel_10792\602977535.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df21['comb'] = new_df21['actor_1_name'] + ' ' + new_df21['actor_2_name'] + ' '+ new_df21['actor_3_name'] + ' '+ new_df21['director_name'] +' ' + new_df21['genres']


,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Roseanne Liang,Chloë Grace Moretz,Taylor John Smith,Beulah Koale,Horror Action War,shadow in the cloud,Chloë Grace Moretz Taylor John Smith Beulah Ko...
1,Anthony Leone,Amy Cay,Brian Patrick Butler,Michael C. Burgess,Adventure Family TV Movie Western,hacksaw,Amy Cay Brian Patrick Butler Michael C. Burges...
2,Yaniv Raz,Lucas Jade Zumann,Taylor Russell,Chase Stokes,Comedy Drama,dr. bird's advice for sad poets,Lucas Jade Zumann Taylor Russell Chase Stokes ...
3,Ramin Bahrani,Adarsh Gourav,Rajkummar Rao,Priyanka Chopra Jonas,Drama,the white tiger,Adarsh Gourav Rajkummar Rao Priyanka Chopra Jo...
4,Doug Liman,Anne Hathaway,Chiwetel Ejiofor,Stephen Merchant,Comedy Crime Romance,locked down,Anne Hathaway Chiwetel Ejiofor Stephen Merchan...


### Data 2022

In [32]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2022" # fetching all the 2022 movies from wikipedia
df5 = pd.read_html(link, header=0)[2]
df6 = pd.read_html(link, header=0)[3]
df7 = pd.read_html(link, header=0)[4]
df8 = pd.read_html(link, header=0)[5]

In [33]:
df9 = pd.concat([df5,df6,df7,df8],ignore_index=True) #concat all the dataframes in a single for preprocessing
df9

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,J A N U A R Y,7.0,The 355,Universal Pictures / Freckle Films / FilmNatio...,Simon Kinberg (director/screenplay); Theresa R...,[2]
1,J A N U A R Y,7.0,The Legend of La Llorona,Saban Films / Ageless Pictures,Patricia Harris Seeley (director); José Prende...,[3]
2,J A N U A R Y,7.0,The Commando,Saban Films / Premiere Entertainment,Asif Akbar (director); Koji Steven Sakai (scre...,[4]
3,J A N U A R Y,7.0,American Siege,Vertical Entertainment,Edward John Drake (director/screenplay); Timot...,[5]
4,J A N U A R Y,14.0,Scream,Paramount Pictures / Spyglass Media Group / Ra...,"Matt Bettinelli-Olpin, Tyler Gillett (director...",[6]
...,...,...,...,...,...,...
316,D E C E M B E R,30.0,"Alice, Darling",Lionsgate / Elevation Pictures,Mary Nighy (director); Alanna Francis (screenp...,[265]
317,D E C E M B E R,NaN,NaN,NaN,NaN,NaN
318,D E C E M B E R,NaN,NaN,NaN,NaN,NaN
319,D E C E M B E R,NaN,NaN,NaN,NaN,NaN


In [44]:
df9 = df9.drop(columns='Ref.',axis='columns')

In [47]:
df9.dropna(inplace= True)

In [48]:
df9.isna().sum()

Opening               0
Opening.1             0
Title                 0
Production company    0
Cast and crew         0
dtype: int64

In [49]:
df9['genres']= df9['Title'].map(lambda x : get_genre(x))

df9

,Opening,Opening.1,Title,Production company,Cast and crew,genres
0,J A N U A R Y,7.0,The 355,Universal Pictures / Freckle Films / FilmNatio...,Simon Kinberg (director/screenplay); Theresa R...,Action Adventure Thriller
1,J A N U A R Y,7.0,The Legend of La Llorona,Saban Films / Ageless Pictures,Patricia Harris Seeley (director); José Prende...,Family Animation Fantasy Horror
2,J A N U A R Y,7.0,The Commando,Saban Films / Premiere Entertainment,Asif Akbar (director); Koji Steven Sakai (scre...,Action Crime Thriller
3,J A N U A R Y,7.0,American Siege,Vertical Entertainment,Edward John Drake (director/screenplay); Timot...,Action Adventure Thriller
4,J A N U A R Y,14.0,Scream,Paramount Pictures / Spyglass Media Group / Ra...,"Matt Bettinelli-Olpin, Tyler Gillett (director...",Horror Mystery Thriller
...,...,...,...,...,...,...
312,D E C E M B E R,23.0,Whitney Houston: I Wanna Dance with Somebody,TriStar Pictures / Black Label Media,Kasi Lemmons (director); Anthony McCarten (scr...,Music History Drama
313,D E C E M B E R,23.0,The Pale Blue Eye,Netflix / Cross Creek Pictures,Scott Cooper (director/screenplay); Christian ...,Thriller Crime Horror Mystery
314,D E C E M B E R,23.0,Women Talking,Orion Pictures / Plan B Entertainment,Sarah Polley (director/screenplay); Rooney Mar...,Drama
315,D E C E M B E R,29.0,A Man Called Otto,Columbia Pictures / Stage 6 Films / Playtone /...,"Marc Forster (director), David Magee (screenpl...",Comedy Drama


In [50]:
df9[['director_name', 'actor_1_name', 'actor_2_name', 'actor_3_name']] = df9['Cast and crew'].apply(extract_info)
df9.head(7)

,Opening,Opening.1,Title,Production company,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,J A N U A R Y,7.0,The 355,Universal Pictures / Freckle Films / FilmNatio...,Simon Kinberg (director/screenplay); Theresa R...,Action Adventure Thriller,Simon Kinberg,Jessica Chastain,Penélope Cruz,Fan Bingbing
1,J A N U A R Y,7.0,The Legend of La Llorona,Saban Films / Ageless Pictures,Patricia Harris Seeley (director); José Prende...,Family Animation Fantasy Horror,Patricia Harris Seeley,Autumn Reeser,Danny Trejo,Antonio Cupo
2,J A N U A R Y,7.0,The Commando,Saban Films / Premiere Entertainment,Asif Akbar (director); Koji Steven Sakai (scre...,Action Crime Thriller,Asif Akbar,Mickey Rourke,Michael Jai White,None
3,J A N U A R Y,7.0,American Siege,Vertical Entertainment,Edward John Drake (director/screenplay); Timot...,Action Adventure Thriller,Edward John Drake,Timothy V. Murphy,Bruce Willis,Rob Gough
4,J A N U A R Y,14.0,Scream,Paramount Pictures / Spyglass Media Group / Ra...,"Matt Bettinelli-Olpin, Tyler Gillett (director...",Horror Mystery Thriller,Tyler Gillett,Melissa Barrera,Mason Gooding,Jenna Ortega
5,J A N U A R Y,14.0,Hotel Transylvania: Transformania,Amazon Studios / Columbia Pictures / Sony Pict...,"Derek Drymon, Jennifer Kluska (directors); Amo...",Animation Comedy Family Adventure Fantasy,Jennifer Kluska,Andy Samberg,Selena Gomez,Kathryn Hahn
6,J A N U A R Y,14.0,Shattered,Lionsgate / Grindstone Entertainment Group,Luis Prieto (director); David Loughery (screen...,Thriller Horror Crime Mystery,Luis Prieto,Cameron Monaghan,Frank Grillo,Lilly Krug


In [56]:
df9=df9.rename(columns={'Title':'movie_title'})
df9.head(4)

,Opening,Opening.1,movie_title,Production company,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,J A N U A R Y,7.0,The 355,Universal Pictures / Freckle Films / FilmNatio...,Simon Kinberg (director/screenplay); Theresa R...,Action Adventure Thriller,Simon Kinberg,Jessica Chastain,Penélope Cruz,Fan Bingbing
1,J A N U A R Y,7.0,The Legend of La Llorona,Saban Films / Ageless Pictures,Patricia Harris Seeley (director); José Prende...,Family Animation Fantasy Horror,Patricia Harris Seeley,Autumn Reeser,Danny Trejo,Antonio Cupo
2,J A N U A R Y,7.0,The Commando,Saban Films / Premiere Entertainment,Asif Akbar (director); Koji Steven Sakai (scre...,Action Crime Thriller,Asif Akbar,Mickey Rourke,Michael Jai White,None
3,J A N U A R Y,7.0,American Siege,Vertical Entertainment,Edward John Drake (director/screenplay); Timot...,Action Adventure Thriller,Edward John Drake,Timothy V. Murphy,Bruce Willis,Rob Gough


In [57]:
new_df22 = df9[['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]
new_df22.sample(2)

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
261,Harry Bradbeer,Millie Bobby Brown,Henry Cavill,Helena Bonham Carter,Adventure Mystery Crime,Enola Holmes 2
23,Kat Coiro,Jennifer Lopez,Owen Wilson,Maluma,Romance Comedy Music,Marry Me


In [58]:
new_df22.isna().sum()

director_name     2
actor_1_name      0
actor_2_name      9
actor_3_name     30
genres            1
movie_title       0
dtype: int64

In [60]:
new_df22.fillna('unknown',inplace=True)

C:\Users\ARMS\AppData\Local\Temp\ipykernel_10792\589004298.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df22.fillna('unknown',inplace=True)


In [61]:
new_df22.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
dtype: int64

In [62]:
new_df22['movie_title']= new_df22['movie_title'].str.lower()

C:\Users\ARMS\AppData\Local\Temp\ipykernel_10792\4239953512.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df22['movie_title']= new_df22['movie_title'].str.lower()


In [63]:

new_df22['comb'] = new_df22['actor_1_name'] + ' ' + new_df22['actor_2_name'] + ' '+ new_df22['actor_3_name'] + ' '+ new_df22['director_name'] +' ' + new_df22['genres']
new_df22.head()

C:\Users\ARMS\AppData\Local\Temp\ipykernel_10792\4038821842.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df22['comb'] = new_df22['actor_1_name'] + ' ' + new_df22['actor_2_name'] + ' '+ new_df22['actor_3_name'] + ' '+ new_df22['director_name'] +' ' + new_df22['genres']


,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Simon Kinberg,Jessica Chastain,Penélope Cruz,Fan Bingbing,Action Adventure Thriller,the 355,Jessica Chastain Penélope Cruz Fan Bingbing Si...
1,Patricia Harris Seeley,Autumn Reeser,Danny Trejo,Antonio Cupo,Family Animation Fantasy Horror,the legend of la llorona,Autumn Reeser Danny Trejo Antonio Cupo Patrici...
2,Asif Akbar,Mickey Rourke,Michael Jai White,unknown,Action Crime Thriller,the commando,Mickey Rourke Michael Jai White unknown Asif A...
3,Edward John Drake,Timothy V. Murphy,Bruce Willis,Rob Gough,Action Adventure Thriller,american siege,Timothy V. Murphy Bruce Willis Rob Gough Edwar...
4,Tyler Gillett,Melissa Barrera,Mason Gooding,Jenna Ortega,Horror Mystery Thriller,scream,Melissa Barrera Mason Gooding Jenna Ortega Tyl...


In [64]:
my_df = pd.concat([new_df21,new_df22],ignore_index=True)

my_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Roseanne Liang,Chloë Grace Moretz,Taylor John Smith,Beulah Koale,Horror Action War,shadow in the cloud,Chloë Grace Moretz Taylor John Smith Beulah Ko...
1,Anthony Leone,Amy Cay,Brian Patrick Butler,Michael C. Burgess,Adventure Family TV Movie Western,hacksaw,Amy Cay Brian Patrick Butler Michael C. Burges...
2,Yaniv Raz,Lucas Jade Zumann,Taylor Russell,Chase Stokes,Comedy Drama,dr. bird's advice for sad poets,Lucas Jade Zumann Taylor Russell Chase Stokes ...
3,Ramin Bahrani,Adarsh Gourav,Rajkummar Rao,Priyanka Chopra Jonas,Drama,the white tiger,Adarsh Gourav Rajkummar Rao Priyanka Chopra Jo...
4,Doug Liman,Anne Hathaway,Chiwetel Ejiofor,Stephen Merchant,Comedy Crime Romance,locked down,Anne Hathaway Chiwetel Ejiofor Stephen Merchan...
...,...,...,...,...,...,...,...
678,Kasi Lemmons,Naomi Ackie,Stanley Tucci,Ashton Sanders,Music History Drama,whitney houston: i wanna dance with somebody,Naomi Ackie Stanley Tucci Ashton Sanders Kasi ...
679,Scott Cooper,Christian Bale,Harry Melling,Gillian Anderson,Thriller Crime Horror Mystery,the pale blue eye,Christian Bale Harry Melling Gillian Anderson ...
680,Sarah Polley,Rooney Mara,Claire Foy,Jessie Buckley,Drama,women talking,Rooney Mara Claire Foy Jessie Buckley Sarah Po...
681,Marc Forster,Tom Hanks,Mariana Treviño,Rachel Keller,Comedy Drama,a man called otto,Tom Hanks Mariana Treviño Rachel Keller Marc F...


In [65]:
old = pd.read_csv(r'D:\Data Science Pojects\Netflix Recommendation System\data 2020\main_data.csv')

old.sample(2)

,director_name,actor_1_name,actor_2_name,actor_3_name,movie_title,genres,comb
2530,Michael Cimino,Robert De Niro,Meryl Streep,John Savage,the deer hunter,Drama War,Robert De Niro Meryl Streep John Savage Michae...
1107,Paul Feig,Michael Rapaport,William Xifaras,Demián Bichir,the heat,Action Comedy Crime,Michael Rapaport William Xifaras Demián Bichir...


In [66]:
final = pd.concat([old,my_df],ignore_index=True)
final

,director_name,actor_1_name,actor_2_name,actor_3_name,movie_title,genres,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,avatar,Action Adventure Fantasy Sci-Fi,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,pirates of the caribbean: at world's end,Action Adventure Fantasy,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,spectre,Action Adventure Thriller,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,the dark knight rises,Action Thriller,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,star wars: episode vii - the force awakens ...,Documentary,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
6826,Kasi Lemmons,Naomi Ackie,Stanley Tucci,Ashton Sanders,whitney houston: i wanna dance with somebody,Music History Drama,Naomi Ackie Stanley Tucci Ashton Sanders Kasi ...
6827,Scott Cooper,Christian Bale,Harry Melling,Gillian Anderson,the pale blue eye,Thriller Crime Horror Mystery,Christian Bale Harry Melling Gillian Anderson ...
6828,Sarah Polley,Rooney Mara,Claire Foy,Jessie Buckley,women talking,Drama,Rooney Mara Claire Foy Jessie Buckley Sarah Po...
6829,Marc Forster,Tom Hanks,Mariana Treviño,Rachel Keller,a man called otto,Comedy Drama,Tom Hanks Mariana Treviño Rachel Keller Marc F...


In [67]:
final.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
movie_title      0
genres           0
comb             0
dtype: int64

In [68]:
final.drop_duplicates(keep='last',inplace=True)

final

,director_name,actor_1_name,actor_2_name,actor_3_name,movie_title,genres,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,avatar,Action Adventure Fantasy Sci-Fi,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,pirates of the caribbean: at world's end,Action Adventure Fantasy,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,spectre,Action Adventure Thriller,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,the dark knight rises,Action Thriller,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,star wars: episode vii - the force awakens ...,Documentary,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
6826,Kasi Lemmons,Naomi Ackie,Stanley Tucci,Ashton Sanders,whitney houston: i wanna dance with somebody,Music History Drama,Naomi Ackie Stanley Tucci Ashton Sanders Kasi ...
6827,Scott Cooper,Christian Bale,Harry Melling,Gillian Anderson,the pale blue eye,Thriller Crime Horror Mystery,Christian Bale Harry Melling Gillian Anderson ...
6828,Sarah Polley,Rooney Mara,Claire Foy,Jessie Buckley,women talking,Drama,Rooney Mara Claire Foy Jessie Buckley Sarah Po...
6829,Marc Forster,Tom Hanks,Mariana Treviño,Rachel Keller,a man called otto,Comedy Drama,Tom Hanks Mariana Treviño Rachel Keller Marc F...


In [69]:
final.to_csv('last_data.csv',index=False)